## TRAIN.PY

In [1]:
import logging
import os
from datetime import datetime, timedelta
from typing import List

import torch
from omegaconf import OmegaConf
from torch.utils.data import ConcatDataset, DataLoader
from tqdm import tqdm

from models.MiDas import MidasNet
from models.TernausNet import UNet16 

from datasets import BaseDepthDataset, DatasetMode, get_dataset
from datasets.mixed_sampler import MixedBatchSampler
from trainers.trainer import NetTrainer
from util.config_util import recursive_load_config
from util.depth_transform import (
    DepthNormalizerBase,
    get_depth_normalizer,
)
from util.logging_util import config_logging

c:\Users\micha\anaconda3\envs\pytorch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
t_start = datetime.now()
print(f"start at {t_start}")
# -------------------- Arguments --------------------
class argumenty():
    def __init__(self) -> None:
        self.config = './config/test_nyu_laptop.yaml'
        self.resume_run = './test_files/24_09_04-02_27_19-test_nyu_laptop_mixloss/checkpoint/latest'
        self.output_dir = './test_files'
        self.no_cuda = False
        self.exit_after = -1
        self.base_data_dir = "C:/Users/micha/Downloads/nyu_v2-20240826T235024Z-001"
        self.add_datetime_prefix = True
        
args = argumenty()
resume_run = args.resume_run
output_dir = args.output_dir
base_data_dir = args.base_data_dir

# -------------------- Initialization --------------------
# Resume previous run
if resume_run is not None:
    print(f"Resume run: {resume_run}")
    out_dir_run = os.path.dirname(os.path.dirname(resume_run))
    job_name = os.path.basename(out_dir_run)
    # Resume config file
    cfg = OmegaConf.load(os.path.join(out_dir_run, "config.yaml"))
else:
    # Run from start
    cfg = recursive_load_config(args.config)
    # Full job name
    pure_job_name = os.path.basename(args.config).split(".")[0]
    # Add time prefix
    if args.add_datetime_prefix:
        job_name = f"{t_start.strftime('%y_%m_%d-%H_%M_%S')}-{pure_job_name}"
    else:
        job_name = pure_job_name

    # Output dir
    if output_dir is not None:
        out_dir_run = os.path.join(output_dir, job_name)
    else:
        out_dir_run = os.path.join("./output", job_name)
    os.makedirs(out_dir_run, exist_ok=False)

# Other directories
out_dir_dic = {
    'ckpt'  : os.path.join(out_dir_run, "checkpoint"),
    'rec' : os.path.join(out_dir_run, "records"),
}
for key in out_dir_dic.keys():
    if not os.path.exists(out_dir_dic[key]):
        os.makedirs(out_dir_dic[key])

# -------------------- Logging settings --------------------
config_logging(cfg.logging, out_dir=out_dir_run)
logging.debug(f"config: {cfg}")
# -------------------- Device --------------------
cuda_avail = torch.cuda.is_available() and not args.no_cuda
device = torch.device("cuda" if cuda_avail else "cpu")
logging.info(f"device = {device}")

# -------------------- Snapshot of code and config --------------------
if resume_run is None:
    _output_path = os.path.join(out_dir_run, "config.yaml")
    with open(_output_path, "w+") as f:
        OmegaConf.save(config=cfg, f=f)
    logging.info(f"Config saved to {_output_path}")
    # Copy and tar code on the first run
    _temp_code_dir = os.path.join(out_dir_run, "code_tar")
    _code_snapshot_path = os.path.join(out_dir_run, "code_snapshot.tar")
    os.system(
        f"rsync --relative -arhvz --quiet --filter=':- .gitignore' --exclude '.git' . '{_temp_code_dir}'"
    )
    os.system(f"tar -cf {_code_snapshot_path} {_temp_code_dir}")
    os.system(f"rm -rf {_temp_code_dir}")
    logging.info(f"Code snapshot saved to: {_code_snapshot_path}")

# -------------------- Data --------------------
loader_seed = cfg.dataloader.seed
if loader_seed is None:
    loader_generator = None
else:
    loader_generator = torch.Generator().manual_seed(loader_seed)

# Training dataset
depth_transform: DepthNormalizerBase = get_depth_normalizer(
    cfg_normalizer=cfg.depth_normalization
)
train_dataset: BaseDepthDataset = get_dataset(
    cfg.dataset.train,
    base_data_dir=base_data_dir,
    mode=DatasetMode.TRAIN,
    augmentation_args=cfg.augmentation_args,
    depth_transform=depth_transform,
    gt_depth_type=cfg.gt_depth_type
)

if "mixed" == cfg.dataset.train.name:
    dataset_ls = train_dataset
    assert len(cfg.dataset.train.prob_ls) == len(
        dataset_ls
    ), "Lengths don't match: `prob_ls` and `dataset_list`"
    concat_dataset = ConcatDataset(dataset_ls)
    mixed_sampler = MixedBatchSampler(
        src_dataset_ls=dataset_ls,
        batch_size=cfg.dataloader.train_batch_size,
        drop_last=True,
        prob=cfg.dataset.train.prob_ls,
        shuffle=True,
        generator=loader_generator,
    )
    train_loader = DataLoader(
        concat_dataset,
        batch_sampler=mixed_sampler,
        num_workers=cfg.dataloader.num_workers,
        pin_memory=cfg.dataloader.pin_memory, 
    )
else:
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=cfg.dataloader.train_batch_size,
        num_workers=cfg.dataloader.num_workers,
        shuffle=True,
        generator=loader_generator,
        pin_memory=cfg.dataloader.pin_memory,
    )
# Validation dataset
val_dataset: BaseDepthDataset = get_dataset(
    cfg.dataset.val,
    base_data_dir=base_data_dir,
    mode=DatasetMode.EVAL,
    depth_transform=depth_transform,
    gt_depth_type=cfg.gt_depth_type
)
if "mixed" == cfg.dataset.val.name:
    dataset_ls = val_dataset
    assert len(cfg.dataset.val.prob_ls) == len(
        dataset_ls
    ), "Lengths don't match: `prob_ls` and `dataset_list`"
    concat_dataset = ConcatDataset(dataset_ls)
    mixed_sampler = MixedBatchSampler(
        src_dataset_ls=dataset_ls,
        batch_size=cfg.dataloader.val_batch_size,
        drop_last=True,
        prob=cfg.dataset.val.prob_ls,
        shuffle=False,
        generator=loader_generator,
    )
    val_loader = DataLoader(
        concat_dataset,
        batch_sampler=mixed_sampler,
        num_workers=cfg.dataloader.num_workers,
        pin_memory=cfg.dataloader.pin_memory,
    )
else:
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=cfg.dataloader.val_batch_size,
        num_workers=cfg.dataloader.num_workers,
        shuffle=False,
        generator=loader_generator,
        pin_memory=cfg.dataloader.pin_memory,
    )
# Test dataset
test_loaders: List[DataLoader] = []
for _test_dic in cfg.dataset.test:
    _test_dataset = get_dataset(
        _test_dic,
        base_data_dir=base_data_dir,
        mode=DatasetMode.EVAL,
        depth_transform=depth_transform,
        gt_depth_type=cfg.gt_depth_type
    )
    _test_loader = DataLoader(
        dataset=_test_dataset,
        batch_size=cfg.dataloader.test_batch_size,
        shuffle=False,
        num_workers=cfg.dataloader.num_workers,
        pin_memory=cfg.dataloader.pin_memory,
    )
    test_loaders.append(_test_loader)

dataloaders = {
    'train' : train_loader,
    'val' : val_loader,
    'tests' : test_loaders,
}

# -------------------- Model --------------------
if cfg.model.name == 'TernausNet':
    model = UNet16(pretrained=True, is_deconv=True)
elif cfg.model.name == 'MiDas':
  model = MidasNet(backbone=cfg.model.backbone)
else:
  raise NotImplementedError 

# -------------------- Trainer --------------------
# Exit time
if args.exit_after > 0:
    t_end = t_start + timedelta(minutes=args.exit_after)
    logging.info(f"Will exit at {t_end}")
else:
    t_end = None

logging.debug(f"Trainer: treiner_nets")
trainer = NetTrainer(
    cfg=cfg,
    model=model,
    dataloaders=dataloaders,
    device=device,
    out_dir_dic=out_dir_dic,
)

# -------------------- Checkpoint --------------------
if resume_run is not None:
    trainer.load_checkpoint(
        resume_run, load_trainer_state=True, resume_lr_scheduler=True
    )

# -------------------- Training & Evaluation Loop --------------------
try:
    trainer.train_and_validate(t_end=t_end)
except Exception as e:
    logging.exception(e)

## CONF / DEVICE

In [ ]:
from util.config_util import recursive_load_config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

conf_path = './config/test_nyu_laptop.yaml'
cfg = recursive_load_config(conf_path)

## TEST LOADERS

In [ ]:
from datasets import DatasetMode, get_dataset
from torch.utils.data import DataLoader
from util.depth_transform import (
    DepthNormalizerBase,
    get_depth_normalizer,
)

depth_transform: DepthNormalizerBase = get_depth_normalizer(
    cfg_normalizer=cfg.depth_normalization
)

test_loaders = []
for _test_dic in cfg.dataset.test:
    _test_dataset = get_dataset(
        _test_dic,
        base_data_dir='C:/Users/micha/Downloads/nyu_v2-20240826T235024Z-001',
        mode=DatasetMode.EVAL,
        gt_depth_type=cfg.gt_depth_type,
        depth_transform=depth_transform,
    )
    _test_loader = DataLoader(
        dataset=_test_dataset,
        batch_size=cfg.dataloader.test_batch_size,
        shuffle=False,
        num_workers=cfg.dataloader.num_workers,
        pin_memory=cfg.dataloader.pin_memory,
    )
    test_loaders.append(_test_loader)

## MODEL

### build

In [ ]:
# -------------------- Model --------------------
if cfg.model.name == 'TernausNet':
    model = UNet16(pretrained=True, is_deconv=True)
elif cfg.model.name == 'MiDas':
  model = MidasNet(backbone=cfg.model.backbone)
else:
  raise NotImplementedError

### load

#### best

In [ ]:
import glob

path = glob.glob('./test_files/*'+conf_path.split('/')[-1].split('.')[-2]+'/checkpoint/latest/best_net.pth')
model.load_state_dict(
    torch.load(path[0], map_location=device)
)
model.to(device)

#### latest

In [ ]:
import glob

path = glob.glob('./test_files/*'+conf_path.split('/')[-1].split('.')[-2]+'/checkpoint/latest/net.pth')
model.load_state_dict(
    torch.load(path[0], map_location=device)
)
model.to(device)

## WIZUALIZACJA

#### best output comp

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
from util.config_util import recursive_load_config
from models.MiDas import MidasNet
from models.TernausNet import UNet16
from util.alignment import align_depth_least_square


def imgshow(inp, title=None, pred=False):
    inp = inp.numpy().transpose((1, 2, 0))
    if not pred:
        mean = np.array([0.48012177, 0.41071795, 0.39187136])
        std = np.array([0.28875302, 0.29516797, 0.30792887])
        inp = std * inp + mean
        inp = np.clip(inp, 0, 1)
        plt.imshow(inp)
    else:
        plt.imshow(inp, cmap='viridis')
    plt.axis("off")
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

config_list = glob.glob('./config/*test_nyu_laptop_lin*.yaml')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, conf in enumerate(config_list):
    cfg = recursive_load_config(conf)
    
    # -------------------- Model --------------------
    if cfg.model.name == 'TernausNet':
        model = UNet16(pretrained=True, is_deconv=True)
    elif cfg.model.name == 'MiDas':
      model = MidasNet(backbone=cfg.model.backbone)
    else:
      raise NotImplementedError
    
    best_path = glob.glob('./test_files/*'+conf.split('\\')[-1].split('.')[-2]+'/checkpoint/latest/best_net.pth')
    model.load_state_dict(
        torch.load(best_path[0], map_location=device)
    )
    model.to(device)
    
    batch = next(iter(test_loaders[0]))
    
    if i == 0:
        fig, (a1,a2) = plt.subplots(1,2)
        a1.imshow(batch['rgb_img'][0][0])
        a2.imshow(batch['depth_raw_linear'][0][0])
        
    pred = model(batch['rgb_img'][0].unsqueeze(0).to(device))
    pred_a = align_depth_least_square(batch['depth_raw_linear'][0].unsqueeze(0), pred.detach().cpu(), batch['valid_mask_raw'][0].unsqueeze(0), return_scale_shift=False)

    fig, (a1,a2) = plt.subplots(1,2)
    a1.imshow(pred.detach().cpu().numpy()[0][0])
    a1.title.set_text('pred_'+conf.split('\\')[-1].split('.')[-2])
    a2.imshow(pred_a[0][0])
    a2.title.set_text('pred_alig_'+conf.split('\\')[-1].split('.')[-2])

    torch.cuda.empty_cache()

#### record comp

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import glob

tr_list = glob.glob('./test_files/*test_nyu_laptop*/*/train*.csv')
vl_list = glob.glob('./test_files/*test_nyu_laptop*/*/val*.csv')


tr0 = pd.read_csv(tr_list[0])


for name in tr0.columns[1:]:
    f, (a1,a2) = plt.subplots(1,2)
    f.suptitle(name)
    for i in range(len(tr_list)):
        tr = pd.read_csv(tr_list[i])
        vl = pd.read_csv(vl_list[i])
        a1.plot(tr['epoch'], tr[name], label = tr_list[i].split('\\')[-3].split('_')[7:])
        a2.plot(vl['epoch'], vl[name], label = vl_list[i].split('\\')[-3].split('_')[7:])
        a1.legend()
        a2.legend()
    plt.show()

# TINCKERING